In [101]:
import sktime
import pandas as pd
from sktime.transformations.panel.rocket import MiniRocket
import statsmodels
from joblib import load

In [102]:
from sktime.datatypes._panel._convert import(
    from_2d_array_to_nested,
    from_nested_to_2d_array,
    is_nested_dataframe
)

In [103]:
model = load('./models/forecaster.joblib')

In [104]:
data = pd.read_csv("./data/BTC-USD.csv",
                    usecols=['Date', 'Close'],
                    parse_dates=['Date'],
                    infer_datetime_format=True)

data = data.set_index('Date').to_period("D")

In [105]:
y = data['Close']

In [106]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
fourier = CalendarFourier(freq="M", order=10)
dp = DeterministicProcess(
    index = data.index,
    constant = True,
    order=3,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)

x = dp.in_sample()
x.shape

(2619, 30)

In [107]:
X_2d = x.values
X_nested = from_2d_array_to_nested(X_2d)
print(f"X is a nested DataFrame: {is_nested_dataframe(X_nested)}")

X is a nested DataFrame: True


In [108]:
minirocket = MiniRocket()
minirocket.fit(X_nested)

MiniRocket()

In [109]:
pred = dp.out_of_sample(steps=50)
pred.index.name = 'date'
pred.shape

(50, 30)

In [110]:
pred_2d = pred.values
nested_pred = from_2d_array_to_nested(pred_2d)
print(f"Pred is a nested DataFrame: {is_nested_dataframe(nested_pred)}")

Pred is a nested DataFrame: True


In [111]:
pred_transform = minirocket.transform(nested_pred)
pred_transform.shape

(50, 9996)

In [115]:
preds = []
final_pred = model.predict(pred_transform)
final_pred[final_pred<0] = 0.0
for pred in range(len(final_pred)):
    preds.append(round(final_pred[pred],2))
print(preds)

[40528.34, 38841.12, 41894.27, 43583.38, 41729.65, 39648.54, 42900.0, 42225.38, 39484.89, 40799.29, 35357.33, 43516.71, 41485.97, 37859.49, 32675.83, 33389.88, 34956.87, 36589.94, 35166.76, 33111.16, 37685.43, 42054.35, 38832.42, 38467.45, 43175.94, 39772.33, 41237.16, 37357.69, 40686.59, 35975.3, 36909.31, 39727.03, 35850.4, 39672.92, 40060.78, 36090.74, 38315.31, 41413.17, 36959.19, 45776.41, 38321.95, 41430.91, 43046.4, 41581.09, 38366.07, 33042.68, 34142.18, 34970.32, 34900.41, 36646.75]
